In [1]:
from AMRGraph import AMR
from AMRData import CustomizedAMR
from utilities import pretty_print, generate_action_sequence, generate_custom_amr
import ActionSequenceGenerator

# AMR - basic example with no exceptions

This is a basic example of AMR parsing and action sequence generation. This means that it does not contain any exception such as coreference, named entities, etc.
Sentence: It looks like we will also bring in whales .

In [2]:
sentence = "It looks like we will also bring in whales ."
amr = AMR.parse_string("""(l / look-02~e.1
      :ARG1~e.2 (b / bring-01~e.6
            :ARG0 (w / we~e.3)
            :ARG1~e.7 (w2 / whale~e.8)
            :mod (a / also~e.5)))""")

Node to concepts hold a dict with keys as variables in the nodes (some nodes do not have variables associated - for instance "-" for polarity)

In [3]:
amr.node_to_concepts

{'a': 'also', 'b': 'bring-01', 'l': 'look-02', 'w': 'we', 'w2': 'whale'}

Mappings between nodes and all the aligned tokens: If the nodes don't have a variable (polarity, literals, quantities, interrogatives), they specify both the aligned tokens and the parent in order to uniquely identify them

In [4]:
amr.node_to_tokens

{'a': ['5'], 'b': ['6'], 'l': ['1'], 'w': ['3'], 'w2': ['8']}

Mappings between relations and tokens. Uniquely identified by also specifying the parent of that relation.
Since the clean-up of parents which are not actual variables is done at the final, we might end up having parents such as 9~e.15 for the relations. However, as I've seen so far, these kind of nodes are usually leaves so hopefully we won't have this problem.

In [5]:
amr.relation_to_tokens

{'ARG1': [[('7', 'b')], [('2', 'l')]]}

Mappings from a node to each child, along with the relation between them.

In [6]:
pretty_print(amr)

Key: a
Leaf

Key: b
ARG0 -> w
ARG1 -> w2
mod -> a

Key: w2
Leaf

Key: w
Leaf

Key: l
ARG1 -> b



All the nodes in the amr should appear here.

In [7]:
amr.keys()

['a', 'b', 'w2', 'w', 'l']

Finally, we create our custom AMR structure:
The CustomAMR outputs 3 dictionaries: 

Parent_dict: which tells the parent of each node in the structure.

Obs: the root node is also included in the dict but the parent field is left empty.  

Tokens_to_concepts_dict: here the tokens are stored as keys and for each token the associated concept is given. The structure of the dictionary is token ->(variable, concept). 

Obs: for the special cases where there is no variable associated to the concept  (for examen polarity) the variable field is left null. 

Relations_dict: the relations dict is the most complex output in the CustomAMR. The structure of the dictionary is (concept, parent) -> (edge_to_parent, [children], token).
The key is a (concept, parent) tuple in order to solve the problem given by the above mentioned special cases. If only the concept would be kept as a key we wouldn’t be able to differentiate the the node aligned to token X and the node aligned to token Y if the node doesn’t have a unique variable (for example 2 negative polarities in the same graph) 

The list of children is constructed using the output of the original amr parser from CAMR[2]. 

In [8]:
custom_AMR = CustomizedAMR()
custom_AMR.create_custom_AMR(amr)

Custom AMR token to concepts dict

In [9]:
custom_AMR.tokens_to_concepts_dict

{1: ('l', 'look-02'),
 3: ('w', 'we'),
 5: ('a', 'also'),
 6: ('b', 'bring-01'),
 8: ('w2', 'whale')}

Custom AMR relations dict

In [10]:
custom_AMR.relations_dict

{('a', 'b'): ('mod', [], ['5']),
 ('b', 'l'): ('ARG1', ['w', 'w2', 'a'], ['6']),
 ('l', ''): ('', ['b'], ['1']),
 ('w', 'b'): ('ARG0', [], ['3']),
 ('w2', 'b'): ('ARG1', [], ['8'])}

Custom AMR parent dict

In [11]:
custom_AMR.parent_dict

{'a': 'b', 'b': 'l', 'l': '', 'w': 'b', 'w2': 'b'}

With this AMR structure, we can generate the sequence of actions which leads from the sentence to the final AMR.

In [12]:
ActionSequenceGenerator.generate_action_sequence(custom_AMR, sentence)

['DN',
 'SH_look-02',
 'DN',
 'SH_we',
 'DN',
 'SH_also',
 'SH_bring-01',
 'RL_mod',
 'RL_ARG0',
 'DN',
 'SH_whale',
 'RR_ARG1',
 'RR_ARG1',
 'DN']

# AMR which requires a SWAP operation

We will now experiment in the same manner with more AMR strings. The first one is a sentence which requires a swap, when we have on the stack the following elements [s2, f, h, s]. Swapping f and h enables us to escape the deadlock and finish the AMR.

In [13]:
amr = """(s2 / seem-01~e.1
       :ARG1~e.3 (h / have-03~e.8
             :ARG0 (w / we~e.7)
             :ARG1 (s / scheme~e.10
                   :mod (p / plan-01~e.15
                   :ARG1 (r / renovate-01~e.14)
                         :ARG1-of (m / major-02~e.13))
                   :purpose (f / future~e.5))))"""
sentence = """It seems that for the future , we have a scheme , a major renovation plan ."""

In [14]:
sequence = generate_action_sequence(generate_custom_amr(AMR.parse_string(amr)), sentence)


Mappings between node variables and their corresponding concepts.

{'f': 'future', 'h': 'have-03', 'm': 'major-02', 'p': 'plan-01', 's': 'scheme', 'r': 'renovate-01', 'w': 'we', 's2': 'seem-01'}

Mappings between nodes and all the aligned tokens: If the nodes don't havea variable (polarity, literals, quantities, interrogatives), they specify both the aligned tokens and the parent in order to uniquely identify them

{'f': ['5'], 'h': ['8'], 'm': ['13'], 'p': ['15'], 's': ['10'], 'r': ['14'], 'w': ['7'], 's2': ['1']}

Mappings between relations and tokens. Uniquely identified by also specifying the parent of that relation.

{'ARG1': [[('3', 's2')]]}

Mappings from a node to each child, along with the relation between them.

Key: f
Leaf

Key: h
ARG0 -> w
ARG1 -> s

Key: m
Leaf

Key: p
ARG1 -> r
ARG1-of -> m

Key: s
purpose -> f
mod -> p

Key: r
Leaf

Key: w
Leaf

Key: s2
ARG1 -> h


All the nodes in the amr should appear here.

['f', 'h', 'm', 'p', 's', 'r', 'w', 's2']

Creating custom A

(s2 / seem-01~e.1  

    :ARG1~e.3 (h / have-03~e.8  
             :ARG0 (w / we~e.7)   
             :ARG1 (s / scheme~e.10   
                   :mod (p / plan-01~e.15  
                         :ARG1 (r / renovate-01~e.14)  
                         :ARG1-of (m / major-02~e.13))  
                   :purpose (f / future~e.5))))  
**It seems that for the future , we have a scheme , a major renovation plan .  **                

In [15]:
sequence

['DN',
 'SH_seem-01',
 'DN',
 'DN',
 'DN',
 'SH_future',
 'DN',
 'SH_we',
 'SH_have-03',
 'RL_ARG0',
 'DN',
 'SH_scheme',
 'DN',
 'DN',
 'SH_major-02',
 'SH_renovate-01',
 'SH_plan-01',
 'RL_ARG1',
 'RL_ARG1-of',
 'RR_mod',
 'DN',
 'SW',
 'RL_purpose',
 'RR_ARG1',
 'RR_ARG1']

# Another AMR which requires a SWAP operation

In [16]:
amr = """(r / recommend-01~e.1
       :ARG1 (a / advocate-01~e.4
             :ARG1 (i / it~e.0)
             :manner~e.2 (v / vigorous~e.3)))"""
sentence = """It should be vigorously advocated ."""

In [17]:
sequence = generate_action_sequence(generate_custom_amr(AMR.parse_string(amr)), sentence)


Mappings between node variables and their corresponding concepts.

{'i': 'it', 'a': 'advocate-01', 'r': 'recommend-01', 'v': 'vigorous'}

Mappings between nodes and all the aligned tokens: If the nodes don't havea variable (polarity, literals, quantities, interrogatives), they specify both the aligned tokens and the parent in order to uniquely identify them

{'i': ['0'], 'a': ['4'], 'r': ['1'], 'v': ['3']}

Mappings between relations and tokens. Uniquely identified by also specifying the parent of that relation.

{'manner': [[('2', 'a')]]}

Mappings from a node to each child, along with the relation between them.

Key: i
Leaf

Key: a
ARG1 -> i
manner -> v

Key: r
ARG1 -> a

Key: v
Leaf


All the nodes in the amr should appear here.

['i', 'a', 'r', 'v']

Creating custom AMR.


Custom AMR token to concepts dict

{0: ('i', 'it'), 1: ('r', 'recommend-01'), 3: ('v', 'vigorous'), 4: ('a', 'advocate-01')}

Custom AMR relations dict

{('i', 'a'): ('ARG1', [], ['0']), ('r', ''): ('', ['a'], [

In [18]:
sequence

['SH_it',
 'SH_recommend-01',
 'DN',
 'SH_vigorous',
 'SH_advocate-01',
 'RL_manner',
 'DN',
 'SW',
 'RL_ARG1',
 'RR_ARG1']

# AMR with aligned polarity (and SWAP)

In [19]:
amr = """(a / allow-01~e.4 :polarity~e.0 -~e.0 
      :ARG1 (e / enter-01~e.6 
            :ARG0 (m / media~e.1) 
            :purpose~e.7 (p / photograph-01~e.8)))"""
sentence = "No media have been allowed to enter for photos ."
sequence = generate_action_sequence(generate_custom_amr(AMR.parse_string(amr)), sentence)


Mappings between node variables and their corresponding concepts.

{'a': 'allow-01', 'p': 'photograph-01', 'm': 'media', 'e': 'enter-01'}

Mappings between nodes and all the aligned tokens: If the nodes don't havea variable (polarity, literals, quantities, interrogatives), they specify both the aligned tokens and the parent in order to uniquely identify them

{'a': ['4'], 'p': ['8'], '-': [('0', 'a')], 'm': ['1'], 'e': ['6']}

Mappings between relations and tokens. Uniquely identified by also specifying the parent of that relation.

{'polarity': [[('0', 'a')]], 'purpose': [[('7', 'e')]]}

Mappings from a node to each child, along with the relation between them.

Key: a
polarity -> -
ARG1 -> e

Key: e
ARG0 -> m
purpose -> p

Key: m
Leaf

Key: -
Leaf

Key: p
Leaf


All the nodes in the amr should appear here.

['a', 'e', 'm', '-', 'p']

Creating custom AMR.


Custom AMR token to concepts dict

{8: ('p', 'photograph-01'), 0: ('-', '-'), 4: ('a', 'allow-01'), 6: ('e', 'enter-01'), 1: ('m'

(a / allow-01~e.4 :polarity~e.0 -~e.0   

    :ARG1 (e / enter-01~e.6   
            :ARG0 (m / media~e.1)   
            :purpose~e.7 (p / photograph-01~e.8)))  

**No media have been allowed to enter for photos .**

In [20]:
sequence

['SH_-',
 'SH_media',
 'DN',
 'DN',
 'SH_allow-01',
 'DN',
 'SH_enter-01',
 'DN',
 'SH_photograph-01',
 'RR_purpose',
 'DN',
 'SW',
 'RL_ARG0',
 'RR_ARG1',
 'RL_polarity']

# Another polarity example (wait for fix of custom AMR to run this example)

In [21]:
amr = """(c / cease-01~e.5 
      :ARG0 (m / metropolis~e.2 
            :mod (t / this~e.0) 
            :mod (d / dynamic~e.1)) 
      :ARG1 (m2 / move-01~e.7 :polarity~e.4 -~e.4))"""
sentence = """This dynamic metropolis does not cease the movement ."""

In [22]:
# sequence = generate_action_sequence(amr, sentence)

# Interrogative example

In [23]:
amr = """(d / differ-02~e.5 :mode~e.11 interrogative~e.11 
      :ARG1 (t / they~e.8) 
      :ARG2 (t2 / they~e.1,8 
            :time (n / now~e.10)))"""
sentence = """Are they going to be different from what they are now ?"""

In [24]:
# sequence = generate_action_sequence(generate_custom_amr(AMR.parse_string(amr)), sentence)

# An example in which an extra DN is performed

In [25]:
import NamedEntityReplacer

amr = """(k / know-01~e.3 
      :ARG1 (r / rich~e.6 
            :domain~e.2,5 (p / person :wiki "Bill_Gates" 
                  :name (n / name :op1 "Bill"~e.0 :op2 "Gates"~e.1))))"""
sentence = """Bill Gates is known to be rich ."""
(new_amr, new_sentence, _) = NamedEntityReplacer.replace_named_entities(AMR.parse_string(amr), sentence)
sequence = generate_action_sequence(generate_custom_amr(new_amr), new_sentence)


Mappings between node variables and their corresponding concepts.

{'p': 'person', 'k': 'know-01', 'r': 'rich'}

Mappings between nodes and all the aligned tokens: If the nodes don't havea variable (polarity, literals, quantities, interrogatives), they specify both the aligned tokens and the parent in order to uniquely identify them

{'p': [0], 'k': ['2'], 'r': ['5']}

Mappings between relations and tokens. Uniquely identified by also specifying the parent of that relation.

{'domain': [[('2', 'r')], ('5', 'r')]}

Mappings from a node to each child, along with the relation between them.

Key: k
ARG1 -> r

Key: p
Leaf

Key: r
domain -> p


All the nodes in the amr should appear here.

['k', 'p', 'r']

Creating custom AMR.


Custom AMR token to concepts dict

{0: ('p', 'person'), 2: ('k', 'know-01'), 5: ('r', 'rich')}

Custom AMR relations dict

{('k', ''): ('', ['r'], ['2']), ('r', 'k'): ('ARG1', ['p'], ['5']), ('p', 'r'): ('domain', [], [0])}

Custom AMR parent dict

{'p': 'r', 'k': '

In [26]:
sequence

['SH_person',
 'DN',
 'SH_know-01',
 'DN',
 'DN',
 'SH_rich',
 'DN',
 'SW',
 'RL_domain',
 'RR_ARG1']